In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from lmfit import minimize, Minimizer, Parameters, fit_report
from typing import List

from lib.transformations import (euler_matrix,
                                 euler_from_matrix,
                                 scale_from_matrix,
)
from lib.utils import convert_to_homogeneous
from lib.io import read_data_to_df
from lib.camera import Camera

def image_residuals(
    params: Parameters,
    X0: np.ndarray,
    p1: np.array, 
    dist: np.array, 
    weights: np.ndarray = None,
    prior_covariance_scale: float = None,
) -> np.ndarray:
    ''' Compute image residuals
    
    Inputs:
    - x0 (np.ndarray): 3D points in object space
    - p1 (np.ndarray): 2D projections in image space
    - weights (np.ndarray, defult = None): weights (e.g., inverse of a-priori observation uncertainty)
    - prior_covariance_scale (float, default = None): A-priori sigma_0^2

    Return:
    - res (nx1 np.ndarray): Vector of the weighted residuals

    '''
    
    # Get parameters  
    parvals = params.valuesdict()
    rx = parvals['rx']
    ry = parvals['ry']
    rz = parvals['rz']
    tx = parvals['tx']
    ty = parvals['ty']
    tz = parvals['tz']
    fx = parvals['fy']
    fy = parvals['fx']
    cx = parvals['cx']
    cy = parvals['cy']

    # Build extrinsics and intrinsics matrix from parameters
    R = euler_matrix(rx, ry, rz)[:3,:3]
    t = np.array([tx, ty, tz])
    K = np.array([
        [fx, 0., cx],
        [0., fy, cy],
        [0., 0., 1.]
    ])
    
    # Project points and remove non-linear distortions
    rvec, _ = cv2.Rodrigues(R)
    p1_, _ = cv2.projectPoints(np.expand_dims(X0, 1),
                                    rvec, t,
                                    K, dist,
                                    )
    p1_ = p1_[:, 0, :]
    
    # Manual approach
    # ex = np.identity(4)
    # ex[:3,:3] = R
    # ex[0:3, 3:4] = t.reshape(3,1)
    # Convert 3D points to homegeneous
    # X0 = convert_to_homogeneous(X0).T
    # Project points 
    # p1_ = K @ (ex @ X0)
    # # Remove non-linear distortions
    # x1_und_ = cv2.undistortPoints(x1_, K, dist, 
    #                               None, K)[:,0,:]                           

    # Compute residuals
    res = p1 - p1_
    
    # If weigthts are provided, scale residual
    if weights is not None:

        res = res * weights


    return res.flatten()



In [11]:
# Read data
camera = Camera(
    width=6000.,
    height=4000.,
    calib_path='../data/cam_intrinsics.txt'
)
K = camera.K
R = camera.R
t = camera.t
dist = camera.dist

R = np.array(
    [[0.56935076,  0.81892662, -0.07210347],
    [-0.13614594,  0.00743009, -0.99066093],
    [-0.81074287,  0.57385015,  0.11572386]]
)
rot = euler_from_matrix(R)
t = np.array(
    [-158., 108., 58.],
)

targets_image = read_data_to_df(
    '../data/targets_image_IMG_2814.csv',
    delimiter=',',
    header=0,
    col_names=['label', 'x', 'y'],
    index_col=0
)


targets_world = read_data_to_df(
    '../data/targets_world.csv',
    delimiter=',',
    header=0,
    col_names=['label', 'X', 'Y', 'Z'],
    index_col=0
)

# Define Parameters to be optimized
params = Parameters()
params.add('rx', value=rot[0], vary=True)
params.add('ry', value=rot[1], vary=True)
params.add('rz', value=rot[2], vary=True)
params.add('tx', value=t[0], vary=True)
params.add('ty', value=t[1], vary=True)
params.add('tz', value=t[2], vary=True)
params.add('fx', value=K[0, 0], vary=False)
params.add('fy', value=K[1, 1], vary=False)
params.add('cx', value=K[0, 2], vary=False)
params.add('cy', value=K[1, 2], vary=False)

# Define Weights as the inverse of the a-priori standard deviation of each observation
# All the measurements are assumed as independent (Q diagonal)
# Weight matrix must have the same shape as the observation matrix (or vector) X0
uncertainty = np.concatenate((
    np.tile(2e-3, (4, 3)),
    np.tile(3e-2, (2, 3))
),
    axis=0,
)
weights = 1. / uncertainty

# A-priori Sigma_0²: scale of the covariance matrix
sigma0_2 = 1.

# Run Optimization!
result = minimize(image_residuals, params,
                  args=(
                      targets_world.to_numpy(),
                      targets_image.to_numpy()
                  ),
                  kws={'dist': dist},
                  scale_covar=True
                  )
# Run Optimization!
minimizer = Minimizer(
    image_residuals,
    params,
    fcn_args=(
        targets_world.to_numpy(),
        targets_image.to_numpy(),
    ),
    fcn_kws={
        'weights': weights,
        'prior_covariance_scale': sigma0_2,
    },
    scale_covar=False,
)
result = minimizer.minimize(method='leastsq')



# Print result
print('-------------------------------')
print('Optimization report')
print(fit_report(result))

# print('-------------------------------')
# print(result.params.pretty_print())
# print('-------------------------------')
# print('Parameter    Value       Stderr')
# for name, param in out.params.items():
#     print(f'{name:7s} {param.value:11.5f} {param.stderr:11.5f}')


print('-------------------------------')
# print(f'Residuals:\n {result.residual.reshape(-1,3)}')
print('Residuals')
print('     x       y')
for res in result.residual.reshape(-1, 2):
    print(f'{res[0]:8.3f} {res[1]:8.3f}')


Using OPENCV camera model + k3
-------------------------------
Optimization report
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 28
    # data points      = 26
    # variables        = 6
    chi-square         = 44.0517497
    reduced chi-square = 2.20258748
    Akaike info crit   = 25.7089819
    Bayesian info crit = 33.2575611
[[Variables]]
    rx:  1.37581456 +/- 2.9159e-04 (0.02%) (init = 1.371803)
    ry:  0.95008714 +/- 1.5657e-04 (0.02%) (init = 0.94542)
    rz: -0.23276695 +/- 3.8963e-04 (0.17%) (init = -0.2347174)
    tx: -160.663167 +/- 0.03943213 (0.02%) (init = -158)
    ty:  110.534357 +/- 0.08917475 (0.08%) (init = 108)
    tz:  56.1283374 +/- 0.10481489 (0.19%) (init = 58)
    fx:  6619.701 (fixed)
    fy:  6619.701 (fixed)
    cx:  3026.061 (fixed)
    cy:  1889.983 (fixed)
[[Correlations]] (unreported correlations are < 0.100)
    C(tx, ty) = 0.774
    C(ty, tz) = 0.614
    C(rz, ty) = -0.579
    C(rx, ty) = 0.566
    C(rx, tz) = 0.561
  

In [ ]:

# Define Parameters to be optimized
params = Parameters()
params.add('rx', value=rot[0], vary=True)
params.add('ry', value=rot[1], vary=True)
params.add('rz', value=rot[2], vary=True)
params.add('tx', value=t[0], vary=True)
params.add('ty', value=t[1], vary=True)
params.add('tz', value=t[2], vary=True)
params.add('fx', value=K[0, 0], vary=True)
params.add('fy', value=K[1, 1], vary=True)
params.add('cx', value=K[0, 2], vary=True)
params.add('cy', value=K[1, 2], vary=True)

# Run Optimization!
result = minimize(image_residuals, params,
                  args=(
                      targets_world.to_numpy(), 
                      targets_image.to_numpy()
                      ),
                  kws = {'dist': dist},          
                  scale_covar=True
)
